In [12]:
import pandas as pd 
import numpy as np 

In [13]:
title = pd.read_csv('/Users/zouxiling/Documents/CIS550 project/title.principals.tsv', sep='\t')
title.head(10)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm1770680,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


In [14]:
name = pd.read_csv('/Users/zouxiling/Documents/CIS550 project/name.basics.tsv', sep='\t')
name.head(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0045537"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0057345,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0077975,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0083922,tt0050976,tt0050986"
5,nm0000006,Ingrid Bergman,1915,1982,"actress,soundtrack,producer","tt0038109,tt0034583,tt0038787,tt0036855"
6,nm0000007,Humphrey Bogart,1899,1957,"actor,soundtrack,producer","tt0042593,tt0037382,tt0034583,tt0043265"
7,nm0000008,Marlon Brando,1924,2004,"actor,soundtrack,director","tt0047296,tt0068646,tt0070849,tt0078788"
8,nm0000009,Richard Burton,1925,1984,"actor,soundtrack,producer","tt0087803,tt0057877,tt0059749,tt0061184"
9,nm0000010,James Cagney,1899,1986,"actor,soundtrack,director","tt0029870,tt0042041,tt0035575,tt0031867"


### Title principal

In [15]:
title.shape

(55193048, 6)

In [16]:
title.category.unique()

array(['self', 'director', 'cinematographer', 'composer', 'producer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'archive_sound'], dtype=object)

In [17]:
# only include 'director', 'actor', and 'actress'
crew_type = ['director','actor','actress']
title1 = title[title['category'].isin(crew_type)]
title1.shape

(28008178, 6)

In [21]:
title1.job.unique()

array(['\\N', 'co-director', 'film director', ...,
       'series director: "Mossy Bear"',
       'series director: "Catch a Falling Star"',
       'director: for Theatre Roundabout'], dtype=object)

In [24]:
# keep the following columns for future joins
# tconst: movie title id
# nconst: crew id
# category: 'director' or 'actor'/'actress'
# characters: if 'actor'/'actress', the character played
crew = title1[['tconst','nconst','category','characters']]
crew.head(5)

,tconst,nconst,category,characters
1,tt0000001,nm0005690,director,\N
3,tt0000002,nm0721526,director,\N
5,tt0000003,nm0721526,director,\N
9,tt0000004,nm0721526,director,\N
11,tt0000005,nm0443482,actor,"[""Blacksmith""]"


In [35]:
crew.isnull().sum()

tconst        0
nconst        0
category      0
characters    0
dtype: int64

In [38]:
# Replace '\N' with null values
crew.replace('\\N', np.nan, inplace=True)

/var/folders/hc/qwh5cpzd0pj2hzldtn6bk0nr0000gn/T/ipykernel_38500/1927941096.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crew.replace('\\N', np.nan, inplace=True)


In [39]:
# seperate directors and actor/actress into two tabels
director = crew[crew['category']=='director'][['tconst','nconst','category']]
act = crew[crew['category'].isin(['actor','actress'])]

print('The director table has {} rows'.format(director.shape[0]))
print('The actor table has {} rows'.format(act.shape[0]))

The director table has 6386298 rows
The actor table has 21621880 rows


In [42]:
act['characters'] = act['characters'].apply(lambda x: x.strip('[]') if x is not np.nan else x)

/var/folders/hc/qwh5cpzd0pj2hzldtn6bk0nr0000gn/T/ipykernel_38500/1795422179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  act['characters'] = act['characters'].apply(lambda x: x.strip('[]') if x is not np.nan else x)


In [43]:
# check missing values
director.isnull().sum()

tconst      0
nconst      0
category    0
dtype: int64

In [45]:
director.head(5)

,tconst,nconst,category
1,tt0000001,nm0005690,director
3,tt0000002,nm0721526,director
5,tt0000003,nm0721526,director
9,tt0000004,nm0721526,director
13,tt0000005,nm0005690,director


In [44]:
act.isnull().sum()

tconst              0
nconst              0
category            0
characters    4704468
dtype: int64

In [46]:
act.head(5)

,tconst,nconst,category,characters
11,tt0000005,nm0443482,actor,"""Blacksmith"""
12,tt0000005,nm0653042,actor,"""Assistant"""
16,tt0000007,nm0179163,actor,NaN
17,tt0000007,nm0183947,actor,NaN
21,tt0000008,nm0653028,actor,"""Sneezing Man"""


### Name basics

In [47]:
name = pd.read_csv('/Users/zouxiling/Documents/CIS550 project/name.basics.tsv', sep='\t')
name.head(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0045537"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0057345,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0077975,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0083922,tt0050976,tt0050986"
5,nm0000006,Ingrid Bergman,1915,1982,"actress,soundtrack,producer","tt0038109,tt0034583,tt0038787,tt0036855"
6,nm0000007,Humphrey Bogart,1899,1957,"actor,soundtrack,producer","tt0042593,tt0037382,tt0034583,tt0043265"
7,nm0000008,Marlon Brando,1924,2004,"actor,soundtrack,director","tt0047296,tt0068646,tt0070849,tt0078788"
8,nm0000009,Richard Burton,1925,1984,"actor,soundtrack,producer","tt0087803,tt0057877,tt0059749,tt0061184"
9,nm0000010,James Cagney,1899,1986,"actor,soundtrack,director","tt0029870,tt0042041,tt0035575,tt0031867"


In [50]:
name.dtypes

nconst               object
primaryName          object
birthYear            object
deathYear            object
primaryProfession    object
knownForTitles       object
dtype: object

In [51]:
name['birthYear'] = name['birthYear'].apply(lambda x: int(x) if x!='\\N' else np.nan)
name['deathYear'] = name['deathYear'].apply(lambda x: int(x) if x!='\\N' else np.nan)

In [52]:
name.isnull().sum()

nconst                      0
primaryName                 3
birthYear            11804746
deathYear            12165176
primaryProfession     2687073
knownForTitles              0
dtype: int64

In [54]:
# add a white space between each two profession
name['primaryProfession'] = name['primaryProfession'].astype(str).apply(lambda x: ', '.join(x.split(',')))

In [57]:
name.head(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack, actor, miscellaneous","tt0072308,tt0053137,tt0050419,tt0045537"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress, soundtrack","tt0117057,tt0071877,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress, soundtrack, music_department","tt0049189,tt0057345,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor, soundtrack, writer","tt0072562,tt0077975,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer, director, actor","tt0060827,tt0083922,tt0050976,tt0050986"
5,nm0000006,Ingrid Bergman,1915.0,1982.0,"actress, soundtrack, producer","tt0038109,tt0034583,tt0038787,tt0036855"
6,nm0000007,Humphrey Bogart,1899.0,1957.0,"actor, soundtrack, producer","tt0042593,tt0037382,tt0034583,tt0043265"
7,nm0000008,Marlon Brando,1924.0,2004.0,"actor, soundtrack, director","tt0047296,tt0068646,tt0070849,tt0078788"
8,nm0000009,Richard Burton,1925.0,1984.0,"actor, soundtrack, producer","tt0087803,tt0057877,tt0059749,tt0061184"
9,nm0000010,James Cagney,1899.0,1986.0,"actor, soundtrack, director","tt0029870,tt0042041,tt0035575,tt0031867"


In [58]:
# split the knowfortitles
name1 = name.assign(knownForTitles=name['knownForTitles'].str.split(',')).explode('knownForTitles').reset_index(drop=True)

In [59]:
name1.head(10)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack, actor, miscellaneous",tt0072308
1,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack, actor, miscellaneous",tt0053137
2,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack, actor, miscellaneous",tt0050419
3,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack, actor, miscellaneous",tt0045537
4,nm0000002,Lauren Bacall,1924.0,2014.0,"actress, soundtrack",tt0117057
5,nm0000002,Lauren Bacall,1924.0,2014.0,"actress, soundtrack",tt0071877
6,nm0000002,Lauren Bacall,1924.0,2014.0,"actress, soundtrack",tt0038355
7,nm0000002,Lauren Bacall,1924.0,2014.0,"actress, soundtrack",tt0037382
8,nm0000003,Brigitte Bardot,1934.0,NaN,"actress, soundtrack, music_department",tt0049189
9,nm0000003,Brigitte Bardot,1934.0,NaN,"actress, soundtrack, music_department",tt0057345


In [60]:
name1.isnull().sum()

nconst                      0
primaryName                 4
birthYear            19706959
deathYear            20926884
primaryProfession           0
knownForTitles              0
dtype: int64